## Perspective Projection
* When we project an object in the real world onto a sensor
![](img/CameraProjection.png)
  * the image on the sensor is a miniature version of the value in the sensor
  * A particular point X,Y,Z will get mapped onto the sensor as
$$(X', Y', Z') = (X\frac{-d}{Z},Y\frac{-d}{Z},-d)$$
  * This is a non-linear operation, as farther points, get divided by different Zs
  * So the math now gets problematic

### Homogenous Coordinates
* Adding another dimension to resolve the issue
$$(\frac{x}{w}, \frac{y}{w}, \frac{z}{w}) =>
\left[
\begin{array}\\
x \\
y \\
z \\
w \\
\end{array}
\right]$$
* So now we can take the projected values and work the math and not worry about the transformation until we need to render
$$\left[
\begin{array}\\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & \frac{1}{f} & 0 \\
\end{array}
\right]\left[
\begin{array}\\
x \\
y \\
z \\
1 \\
\end{array}
\right] = \left[
\begin{array}\\
x \\
y \\
\frac{z}{f}\\
\end{array}
\right] => (x\frac{f}{z},y\frac{f}{z}) => (u,v)$$
  * The conversion to the projected co-ordinate system can occur only at the very end when we want to render or do a image manipulation operation
  * This is also scale invariant. Multiplying the original operation by a constant does not impact the final output
  
#### Example Implementation

In [21]:
import numpy as np

# Simple projection function
def Homogenous2Image(_v):
    return [_v[0]/_v[2],_v[1]/_v[2]]

def ProjectOntoImage(_P, _f):
    ProjMat = np.zeros((3,4),dtype = np.float)
    ProjMat[0][0] = ProjMat[1][1] = 1.0
    ProjMat[2][2] = (1/_f)
    
    tempP = np.append(_P, 1.0)
    
    return np.matmul(ProjMat, tempP)
    
# Point in the real world in mm
P = [4000.0, 10000.0, 50000.0]
# Focal length
f = 100.0

print ("Real World Co-ordinates: ",P)
print ("Homogenous Co-ordinates: ",ProjectOntoImage(P, f))
print ("Image Co-ordinates: ",Homogenous2Image(ProjectOntoImage(P, f)))


('Real World Co-ordinates: ', [4000.0, 10000.0, 50000.0])
('Homogenous Co-ordinates: ', array([  4000.,  10000.,    500.]))
('Image Co-ordinates: ', [8.0, 20.0])


#### Properties
* A point / line / Polygon on the real world gets mapped to a point / line and polygon on the perspective projection respectively
* Almost all parallel lines in the real world, meet at a "vanishing" point in the projection (think of a road / railway line)
* All lines in the same plane, converge at various vanishing points that are co-linear (as in the vanishing points) line on a single line. This line is the horizon.


## Other projection models
* Orthographic Projection
  * Where we model telephoto lens, where all the rays are coming in parallel to each other
  * So there is no transformation, it's just the exact same image on the sensor and it's scaled (X & Y) down to the sensor size
  * Important thing is that Z is not used
* Weak projection
  * Here we assume that all the real world objects that we want to map is at a constant depth Z<sub>0</sub>
  * Rest remains the same as the Homogenous Perspective
  * This can be used where the X & Y scale of the object is so huge, that the depth variations within the object does not matter